### 베이지안 최적화 개요와 HyperOpt 사용법
- 목적함수, 검색공간 

In [3]:
import hyperopt

print(hyperopt.__version__)

0.2.7


- 검색공간 설정 

In [ ]:
from hyperopt import hp 

# 검색공간 (최적화 파라미터를 입력 시킬 값)
# -10 ~ 10까지 1간격을 가지는 입력 변수 x 집합값과 -15 ~ 15까지 1간격을 가지는 입력 변수  y 집합값 설정.
search_space = {'x' : hp.quniform('x', -10, 10, 1), 'y' : hp.quniform('y', -15, 15, 1)}
search_space

{'x': <hyperopt.pyll.base.Apply at 0x23da1c67290>,
 'y': <hyperopt.pyll.base.Apply at 0x23da1ca4710>}

- 목적함수 설정 

In [5]:
from hyperopt import STATUS_OK

# 목적함수 생성 
# 입력 변수값과 입력 변수 검색 범위를 가지는 딕셔너리를 인자로 받고,  특정 값을 반환 
def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y

    return retval # return {'loss': retval, 'status':STATUS_OK}

In [6]:
from hyperopt import fmin, tpe, Trials
import numpy as np 

# 입력 결괏값을 저장한 Trials 객체값 생성 
trial_val = Trials()

# 목적 함수의 최솟값을 반환하는 최적 입력 변숫값을 5번의 입력값 시도(max_evals=5)로 찾아냄.
best_01 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=5,
               trials=trial_val, rstate=np.random.default_rng(seed=0))

print('best :', best_01)
# rstate : 일반적인 정수형 값 x 

100%|██████████| 5/5 [00:00<00:00, 714.95trial/s, best loss: -224.0]
best : {'x': np.float64(-4.0), 'y': np.float64(12.0)}


In [7]:
trial_val = Trials()

# max_evals를 20회로 늘려서 재테스트
best_02 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=20
               , trials=trial_val, rstate=np.random.default_rng(seed=0))
print('best:', best_02)

100%|██████████| 20/20 [00:00<00:00, 1052.68trial/s, best loss: -296.0]
best: {'x': np.float64(2.0), 'y': np.float64(15.0)}


- HyperOpt 수행시 적용된 입력값들과 목적함수 반환값 보기

In [ ]:
# results속성 : 목적함수 반환값들이 저장 {'loss':함수 반환값, 'status':반환 상태값}
print(trial_val.results)

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -296.0, 'status': 'ok'}, {'loss': -40.0, 'status': 'ok'}, {'loss': 281.0, 'status': 'ok'}, {'loss': 64.0, 'status': 'ok'}, {'loss': 100.0, 'status': 'ok'}, {'loss': 60.0, 'status': 'ok'}, {'loss': -39.0, 'status': 'ok'}, {'loss': 1.0, 'status': 'ok'}, {'loss': -164.0, 'status': 'ok'}, {'loss': 21.0, 'status': 'ok'}, {'loss': -56.0, 'status': 'ok'}, {'loss': 284.0, 'status': 'ok'}, {'loss': 176.0, 'status': 'ok'}, {'loss': -171.0, 'status': 'ok'}, {'loss': 0.0, 'status': 'ok'}]


In [10]:
# vals 속성 : 입력 변수값들 반환 {'입력변수명':개별 수행 시마다 입력된 값 리스트}
print(trial_val.vals)

{'x': [np.float64(-6.0), np.float64(-4.0), np.float64(4.0), np.float64(-4.0), np.float64(9.0), np.float64(2.0), np.float64(10.0), np.float64(-9.0), np.float64(-8.0), np.float64(-0.0), np.float64(-0.0), np.float64(1.0), np.float64(9.0), np.float64(6.0), np.float64(9.0), np.float64(2.0), np.float64(-2.0), np.float64(-4.0), np.float64(7.0), np.float64(-0.0)], 'y': [np.float64(5.0), np.float64(10.0), np.float64(-2.0), np.float64(12.0), np.float64(1.0), np.float64(15.0), np.float64(7.0), np.float64(-10.0), np.float64(0.0), np.float64(-5.0), np.float64(-3.0), np.float64(2.0), np.float64(4.0), np.float64(10.0), np.float64(3.0), np.float64(3.0), np.float64(-14.0), np.float64(-8.0), np.float64(11.0), np.float64(-0.0)]}


In [11]:
import pandas as pd 

# results에서 loss 키값에 해당하는 value들을 추출하여 list로 생성 
losses = [loss_dict['loss'] for loss_dict in trial_val.results]

# DataFrame으로 생성. 
result_df = pd.DataFrame({'x': trial_val.vals['x'],
                         'y': trial_val.vals['y'],
                          'losses': losses
                         }
                        )
result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,2.0,15.0,-296.0
6,10.0,7.0,-40.0
7,-9.0,-10.0,281.0
8,-8.0,0.0,64.0
9,-0.0,-5.0,100.0


### HyperOpt를 XGBoost 하이퍼 파라미터 튜닝에 적용

In [13]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

dataset = load_breast_cancer()

cancer_df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
cancer_df['target'] = dataset.target

X_features = cancer_df.iloc[:, :-1]
y_label = cancer_df.iloc[:, -1]

# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label,
                                         test_size=0.2, random_state=156 )

# 학습 데이터를 다시 학습과 검증 데이터로 분리 
X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train,
                                         test_size=0.1, random_state=156 )

In [ ]:
from hyperopt import hp


# max_depth는 5에서 20까지 1간격으로, min_child_weight는 1에서 2까지 1간격으로
# colsample_bytree는 0.5에서 1사이, learning_rate는 0.01에서 0.2사이 정규 분포된 값으로 검색. 
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
               }

In [17]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

# fmin()에 입력된 검색공간은 모두 실수형 
# XGBClassifier의 정수형 하이퍼 파라미터는 정수형으로 변환 필요 
# 정확도는 높을 수록 더 좋은 수치 
# -1 * 정확도를 곱해서 큰 정확도 값일 수록 최소가 되도록 변환 0.8 < 0.9 (fmin()은 최솟값을 최적화함) -> -0.8 > -0.9 
def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'], 
                            eval_metric='logloss')
    
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)

    # accuracy는 cv=3 개수만큼의 정확도 결과를 가지므로 이를 평균해서 반환하되 -1을 곱해줌. 
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}
    

In [18]:
from hyperopt import fmin, tpe, Trials

trial_val = Trials()
best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', best)

100%|██████████| 50/50 [00:12<00:00,  4.09trial/s, best loss: -0.967047170907401] 
best: {'colsample_bytree': np.float64(0.7923560109541058), 'learning_rate': np.float64(0.1938940946651398), 'max_depth': np.float64(16.0), 'min_child_weight': np.float64(2.0)}


In [19]:
print('colsample_bytree:{0}, learning_rate:{1}, max_depth:{2}, min_child_weight:{3}'.format(
                        round(best['colsample_bytree'], 5), round(best['learning_rate'], 5),
                        int(best['max_depth']), int(best['min_child_weight'])))

colsample_bytree:0.79236, learning_rate:0.19389, max_depth:16, min_child_weight:2


- 위의 최적화 하이퍼 파라미터로 재학습 

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [23]:
xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=round(best['learning_rate'], 5), 
                            max_depth=int(best['max_depth']), min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5), early_stopping_rounds=50, eval_metric='logloss'
                           )

evals = [(X_tr, y_tr), (X_val, y_val)]
xgb_wrapper.fit(X_tr, y_tr, 
                eval_set=evals, verbose=True)

preds = xgb_wrapper.predict(X_test)
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, preds, pred_proba)

[0]	validation_0-logloss:0.52017	validation_1-logloss:0.56664
[1]	validation_0-logloss:0.41762	validation_1-logloss:0.49501
[2]	validation_0-logloss:0.34123	validation_1-logloss:0.43295
[3]	validation_0-logloss:0.28440	validation_1-logloss:0.39091
[4]	validation_0-logloss:0.24056	validation_1-logloss:0.35565
[5]	validation_0-logloss:0.20676	validation_1-logloss:0.33910
[6]	validation_0-logloss:0.17704	validation_1-logloss:0.31555
[7]	validation_0-logloss:0.15563	validation_1-logloss:0.29982
[8]	validation_0-logloss:0.13628	validation_1-logloss:0.28915
[9]	validation_0-logloss:0.11984	validation_1-logloss:0.27626
[10]	validation_0-logloss:0.10728	validation_1-logloss:0.27569
[11]	validation_0-logloss:0.09639	validation_1-logloss:0.27079
[12]	validation_0-logloss:0.08691	validation_1-logloss:0.26944
[13]	validation_0-logloss:0.07998	validation_1-logloss:0.26511
[14]	validation_0-logloss:0.07359	validation_1-logloss:0.26765
[15]	validation_0-logloss:0.06817	validation_1-logloss:0.26790
[1

In [24]:
losses = [loss_dict['loss'] for loss_dict in trial_val.results]
result_df = pd.DataFrame({'max_depth': trial_val.vals['max_depth'],
                          'min_child_weight': trial_val.vals['min_child_weight'],
                          'colsample_bytree': trial_val.vals['colsample_bytree'],
                          'learning_rate': trial_val.vals['learning_rate'],
                          'losses': losses
                         }
                        )
result_df

,max_depth,min_child_weight,colsample_bytree,learning_rate,losses
0,19.0,2.0,0.585235,0.033688,-0.945117
1,5.0,2.0,0.727186,0.105956,-0.962676
2,6.0,2.0,0.959945,0.154804,-0.960454
3,6.0,2.0,0.950012,0.120686,-0.960483
4,16.0,2.0,0.674336,0.142392,-0.960454
5,8.0,2.0,0.863774,0.106579,-0.960454
6,14.0,2.0,0.957521,0.079111,-0.956082
7,19.0,2.0,0.695018,0.095213,-0.960483
8,9.0,2.0,0.684442,0.147520,-0.960483
9,8.0,1.0,0.592116,0.081179,-0.953904
